In [1]:
import openai
openai.api_key = ""

In [38]:
text = """Многие военные и политические эксперты оценивают нынешнюю обстановку в мире, как крайне опасную и острую, в первую очередь из-за угрозы применения ядерного оружия. За последние десятилетия еще не было столь высокого риска развязывания войны с применением подобного вооружения как сейчас.
Доцент кафедры политического анализа и социально-психологических процессов РЭУ им. Плеханова Александр Перенджиев в колонке для ФАН обратил внимание, что за сложившуюся ситуацию в мире в ответе США и страны Запада. Именно эти государства разрушили равноправную международную систему безопасности. На фоне этого политолог предложил свой метод борьбы с агрессией недружественных государств — через формирование антизападного ядерного союза.
В этот раз я хотел бы поговорить о необходимости создания антизападного ядерного союза с участием России. В настоящее время, по сути, разрушена прежде всего правовая система обеспечения международной безопасности в мире. Мы помним выход Штатов из ПРО (Договор об ограничении систем противоракетной обороны), помним их выходы и из других договоров. Даже Договор о сокращении стратегических наступательных вооружений Россия была вынуждена заморозить. Все по причине агрессивных действий США."""

type_prompt = "Определи, является ли текст: новостью, статьёй, записью в личном блоге, публичным комментарием или личным сообщением. Верни только категорию текста. Если текст не относится ни одной из этих категорий, верни 'другое'."

system_content = "Ты - журналист и исследователь российских СМИ, работающий в условиях диктатуры и военной цензуры (война между Россией и Украиной, с вмешательством Европы и США)."
references_prompt = """Есть ли в тексте ссылки на источники информации? Если они отсутствуют, верни только "Источники: не обнаружены."

Текст:
"""
hatespeech_prompt = """Проверь текст на наличие дискриминирующих, стигматизирующих или провоцирующих ненависть лексики, высказываний или идей (например, трансфобию, гомофобию, национализм, расизм или сексизм).

Текст:
"""

manipulation_prompt = """Извлеки все приёмы манипуляции читателем из следующего текста и их краткое обоснование в виде пронумерованного списка. Если в тексте нет приёмов манипуляции, верни "Манипуляция: не обнаружена".

Текст:
"""

logic_err_prompt = """Проверь текст (внутри тройной `) на наличие логических ошибок или когнитивных искажений. Если они не обнаружены в тексте, напиши, что "Логические ошибки: не обнаружены". Если они, то сформулируй свой ответ так:
1) Наличие логических ошибок: поочередно перечисли название логических ошибок и для каждой из них кратко (одно предложение) объясни на примере текста, почему это логическая ошибка.
2) Поправка: кратко (в двух предложениях) объясни, почему обнаруженные логические ошибки опасны, и как их избежать.

Например:

Наличие логических ошибок:
1) Атака личности. Уоррен атакует личность Уилла, обвиняя его в ненависти к стране, вместо того чтобы обсуждать его аргументы по поводу военных расходов.
2) Подмена тезиса: Уоррен искажает слова Уилла и потом опровергает своё искажение.

Поправка: Обнаруженные логические ошибки опасны, так как отвлекают от рассмотрения фактов и аргументов, и могут вести к искаженному восприятию ситуации. Чтобы избежать их, важно сосредотачиваться на деле и обсуждении аргументов, а не переходить на личности собеседников.
"""

In [45]:
def process_text(text):
    run_analysis = False
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": f"{type_prompt} Текст: '{text}'"}],
      temperature=0,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    text_type = response['choices'][0]['message']['content']
    if text_type.lower() != 'личное сообщение':
        run_analysis = True
    if run_analysis:
        return analyse_text(text)
    else:
        return {}
    
def analyse_text(text):
    field2prompt = {'manipulation_methods': manipulation_prompt,
               'hatespeech': hatespeech_prompt,
               'references': references_prompt}
    criteria = {field: None for field in field2prompt.keys()}
    for field_name, prompt in field2prompt.items():
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[{"role": "system", "content": system_content},
                    {"role": "user", "content": f"{prompt} '{text}'"}],
          temperature=0,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        answer = response['choices'][0]['message']['content']
        criteria[field_name] = answer
    if criteria['references'] == "Источники: не обнаружены.":
        criteria['references_present'] = 0
    else:
        criteria['references_present'] = 1
    if "Манипуляция: не обнаружена" in criteria['manipulation_methods']:
        criteria['manipulation_present'] = 0
    else:
        criteria['manipulation_present'] = 1
    return criteria

In [46]:
process_text(text)

{'manipulation_methods': '1. Манипуляция: создание чувства угрозы и опасности.\nОбоснование: Автор текста утверждает, что нынешняя обстановка в мире является крайне опасной и острой из-за угрозы применения ядерного оружия. Это создает чувство страха и тревоги у читателя, что может влиять на его мнение и восприятие ситуации.\n\n2. Манипуляция: использование авторитета эксперта.\nОбоснование: Автор текста ссылается на мнение доцента кафедры политического анализа и социально-психологических процессов РЭУ им. Плеханова, чтобы подтвердить свою точку зрения. Это создает впечатление, что мнение автора подкреплено авторитетным и независимым экспертом, что может убедить читателя принять его точку зрения.\n\n3. Манипуляция: упрощение и черно-белое мышление.\nОбоснование: Автор текста утверждает, что за сложившуюся ситуацию в мире в ответе только США и страны Запада, которые разрушили равноправную международную систему безопасности. Такое упрощение и черно-белое мышление не учитывает сложности и 

In [47]:
process_text("Привет! Как дела?")

{}